In [5]:
from httparse import RequestParser
from httptools import HttpRequestParser

In [6]:
buff = b"GET /cookies/foo/bar/baz?a=1&b=2 HTTP/1.1\r\nHost: 127.0.0.1:8090\r\nConnection: keep-alive\r\nCache-Control: max-age=0\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nUser-Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.56 Safari/537.17\r\nAccept-Encoding: gzip,deflate,sdch\r\nAccept-Language: en-US,en;q=0.8\r\nAccept-Charset: ISO-8859-1,utf-8;q=0.7,*;q=0.3\r\nCookie: name=wookie\r\n\r\n"

In [7]:
parser = RequestParser()
%timeit parser.parse(buff)

1.08 µs ± 6.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
class Proto:
    __slots__ = ()
    def on_headers_complete(self):
        ...

proto = Proto()

%timeit HttpRequestParser(proto).feed_data(buff)

2 µs ± 63.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
